In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from importlib import reload
import matplotlib.pyplot as plt

import utils.models as models
import utils.plotting as plotting
import utils.dataloaders as dl
import utils.traintest as tt
import utils.adversarial as adv
import utils.eval as ev
import model_params as params
import utils.resnet_orig as resnet
import utils.gmm_helpers as gmm_helpers

import model_paths

from tensorboardX import SummaryWriter
%load_ext line_profiler

In [53]:
def show_stats(model, device, model_params):
    acc = tt.test(model, device, model_params.test_loader, min_conf=.001)[0]
    df = ev.evaluate(model, device, model_params.data_name, model_params.loaders)
    return df, acc

In [67]:
dataset = 'FMNIST'
device = torch.device('cuda:6')
model_params = params.params_dict[dataset]()
model_path = model_paths.model_dict[dataset]() 
model_list = [torch.load(file).to(device) for file in model_path.files]

/usr/local/lib/python3.5/dist-packages/torch/serialization.py:435: SourceChangeWarning: source code of class 'torch.nn.modules.conv.Conv2d' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/usr/local/lib/python3.5/dist-packages/torch/serialization.py:435: SourceChangeWarning: source code of class 'torch.nn.modules.batchnorm.BatchNorm2d' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/usr/local/lib/python3.5/dist-packages/torch/serialization.py:435: SourceChangeWarning: source code of class 'torch.nn.modules.linear.Linear' has changed. you can retrieve the original source code by accessing the object's source attribute or set `

In [68]:
accuracies = [tt.test(model, device, model_params.test_loader, min_conf=.001)[0]
              for model in model_list]
results = [ev.evaluate(model, device, model_params.data_name, model_params.loaders) 
           for model in model_list]
test_error = [100*(1.-acc) for acc in accuracies]
keys = [key + ' (TE: ' + str(te) + '\%)' for (te, key) in zip(test_error, model_path.keys)]
df = pd.concat(results, axis=1, keys=keys)

In [69]:
keys = [key + ' (TE: {:.2f}%)'.format(te) for (te, key) in zip(test_error, model_path.keys)]
df = pd.concat(results, axis=1, keys=keys)
df = pd.concat(results, axis=1, keys=keys)

In [70]:
print(df.round(3).to_latex())

\begin{tabular}{lrrrrrrrr}
\toprule
{} & \multicolumn{2}{l}{Base (TE: 4.66\%)} & \multicolumn{2}{l}{CEDA (TE: 5.03\%)} & \multicolumn{2}{l}{ODIN (TE: 4.69\%)} & \multicolumn{2}{l}{GMM (TE: 5.04\%)} \\
{} &              MMC &  AUROC &              MMC &  AUROC &              MMC &  AUROC &             MMC &  AUROC \\
DataSet      &                  &        &                  &        &                  &        &                 &        \\
\midrule
orig         &            0.984 &  0.000 &            0.985 &  0.000 &            0.423 &  0.000 &           0.984 &  0.000 \\
MNIST        &            0.693 &  0.967 &            0.689 &  0.973 &            0.224 &  0.994 &           0.708 &  0.968 \\
EMNIST       &            0.681 &  0.969 &            0.668 &  0.975 &            0.226 &  0.994 &           0.660 &  0.974 \\
GrayCIFAR10  &            0.893 &  0.893 &            0.842 &  0.931 &            0.369 &  0.794 &           0.527 &  0.969 \\
Noise        &            0.781 &  0.9

In [71]:
df.to_csv('results/' + dataset)

Base (TE: 4.66%)           CEDA (TE: 5.03%)            \
                          MMC     AUROC              MMC     AUROC   
DataSet                                                              
orig                 0.984078  0.000000         0.984831  0.000000   
MNIST                0.692679  0.966765         0.689281  0.973178   
EMNIST               0.681222  0.969314         0.668332  0.974855   
GrayCIFAR10          0.893414  0.893392         0.842169  0.930958   
Noise                0.781063  0.958617         0.548391  0.987270   
UniformNoise         0.643546  0.981035         0.750159  0.961726   

             ODIN (TE: 4.69%)           GMM (TE: 5.04%)            
                          MMC     AUROC             MMC     AUROC  
DataSet                                                            
orig                 0.423192  0.000000        0.983714  0.000000  
MNIST                0.224143  0.993511        0.707906  0.968166  
EMNIST               0.226076  0.993593        0.659582  0.974457  
GrayCIFAR10          0.368576  0.793775        0.526969  0.969102  
Noise                0.296943  0.959822        0.718811  0.960331  
UniformNoise         0.195112  0.999312        0.100000  1.000000

In [44]:
reload(params)

<module 'model_params' from '/home/alexm/project/notebooks/gmm-robust/model_params.py'>

In [41]:
model_list[-2].model.temperature

Parameter containing:
tensor(1000., device='cuda:7')

In [46]:
reload(dl)

<module 'utils.dataloaders' from '/home/alexm/project/notebooks/gmm-robust/utils/dataloaders.py'>